# 4. Modelling

## 4.1 Importing useful libraries & functions

In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import pandas as pd
import numpy as np

# processing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

# modelling
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.regularizers import L2
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import RandomizedSearchCV
# evaluating
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

## 4.2 Reading data

In [3]:
cars = pd.read_csv('../data/clean_cars.csv')

In [4]:
cars

,year,cv,km,fuel,doors,gearbox,color,brand,price,boot,...,seats,cmixto,0-100,displac,cylinders,max_par,gear,class,location,area
0,6,326,94000,gasolina,2,automatic,blanco,bmw,35900,445,...,4,6.6,5.0,2998,less4 inline,450,8,sport,rioja+la,84912
1,2,252,29187,gasolina,2,manual,blanco,other,65450,100,...,2,6.4,4.5,1798,4 inline,320,7,sport,rioja+la,75240
2,4,150,40012,gasolina,5,manual,rojo,renault,20950,377,...,5,5.6,9.9,1333,4 inline,220,6,4x4,rioja+la,73336
3,14,224,329000,diesel,5,automatic,negro,mercedes,10900,450,...,5,7.9,7.5,2987,6 V,540,7,4x4,rioja+la,84258
4,1,90,5450,gasolina,5,manual,naranja,renault,23450,536,...,5,5.3,14.0,999,more4 inline,160,6,4x4,rioja+la,76140
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
285217,6,180,51000,diesel,5,automatic,gris,jaguar,36100,565,...,5,4.6,7.1,1999,4 inline,430,8,standard,valencia,98704
285218,5,150,112399,diesel,5,automatic,gris,audi,28700,480,...,5,4.2,8.9,1968,4 inline,320,7,standard,valencia,87032
285219,6,116,62241,gasolina,4,automatic,amarillo,audi,18400,425,...,5,4.6,9.9,999,more4 inline,200,7,standard,valencia,80280
285220,7,69,49137,gasolina,5,manual,blanco,toyota,11155,286,...,5,4.3,15.3,998,more4 inline,93,5,standard,valencia,66130


## 4.3 Train, Test split

We will divide our data into three datasets. The **training** dataset will teach the model how to operate, the **cross-validation (CV)** dataset will help us optimize the parameters of the models, and finally the **test** dataset will be used to evaluate our final mode and estimate its error and accuracy.

In [5]:
X = cars.drop('price',axis=1)
y = cars['price']

# Creating train dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
X_train.reset_index(drop=True,inplace=True)
y_train.reset_index(drop=True,inplace=True)
X_test.reset_index(drop=True,inplace=True)
y_test.reset_index(drop=True,inplace=True)


I split the features into numerical and categorical, so I can later normalize and one-hot encode.

In [6]:
# 1. Training dataset
X_num = X_train.select_dtypes(np.number)
X_cat = X_train.select_dtypes(object)

# 2. Test dataset
X_num_test = X_test.select_dtypes(np.number)
X_cat_test = X_test.select_dtypes(object)

## 4.4 Normalizing numerical features

In [7]:
def std_scale(numericals, transformer):
    normalized = []
    for X_numerical in numericals:
        X_normalized = transformer.transform(X_numerical)
        X_normalized = pd.DataFrame(X_normalized, columns=X_numerical.columns)
        normalized.append(X_normalized)
    return normalized

In [8]:
# Defining normalizer
transformer = StandardScaler().fit(X_num)

# Applying normalizer
X_norm, X_norm_test = std_scale([X_num,X_num_test], transformer)

## 4.5 Encoding categorical features

In [9]:
def one_hot(categoricals, encoder):
    onehots = []
    for X_categorical in categoricals:
        encoded = encoder.transform(X_categorical).toarray()
        onehot_encode = pd.DataFrame(encoded,columns=encoder.get_feature_names_out(X_categorical.columns))
        onehots.append(onehot_encode)
    return onehots

In [10]:
# Creating encoder
encoder = OneHotEncoder(handle_unknown='error',drop='first').fit(X_cat)

# Applying onehot-encode
X_oh, X_oh_test = one_hot([X_cat,X_cat_test], encoder)

## 4.6 Concatenating back

In [11]:
X_train_scaled = pd.concat([X_norm, X_oh], axis=1)
X_test_scaled = pd.concat([X_norm_test, X_oh_test], axis=1)

## 4.7 First-glance at models

In [18]:
model1 = LinearRegression()
model2 = KNeighborsRegressor()
model3 = RandomForestRegressor()
model4 = XGBRegressor()

model_pipeline = [model1, model2, model3, model4]
model_names = ['Linear Regression','KNN','Random Forest', 'XGBoost']

scores = {}

for model, model_name in zip(model_pipeline, model_names):
    mean_score = np.mean(cross_val_score(model, X_train_scaled, y_train, cv=5))
    scores[model_name] = mean_score

In [22]:
print(scores)

{'Linear Regression': 0.8000673647965687, 'KNN': 0.9315656293117109, 'Random Forest': 0.9644366640322858, 'XGBoost': 0.9454349230714945}


In [41]:
# Define the K-fold Cross Validator
kfold = KFold(n_splits=2, shuffle=True)

# K-fold Cross Validation model evaluation
fold_no = 1
scores = []

for cv_train, cv_test in kfold.split(X_train_scaled, y_train):
    model = Sequential(
    [
        tf.keras.layers.Dense(64, activation = 'relu', name='L4'),
        tf.keras.layers.Dense(1, activation = 'relu', name='L5'),

    ])
    model.compile(loss = MeanSquaredError(), optimizer = tf.keras.optimizers.Adam(learning_rate=0.01))
        
    # Generate a print
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')
    
    model.fit(X_train_scaled.loc[cv_train,:], y_train.loc[cv_train], epochs = 5)

    # Generate generalization metrics
    y_pred = model.predict(X_train_scaled.loc[cv_test,:])
    training_error = r2_score(y_pred,y_train.loc[cv_test])
    
    print(f'R2 for fold {fold_no}: {training_error:.3f}')
    scores.append(training_error)

    # Increase fold number
    fold_no = fold_no + 1
print(f'The CV R2 score is {np.mean(scores):.3f}.')

------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/5
3120/3120 [==============================] - 8s 2ms/step - loss: 77793376.0000
Epoch 2/5
3120/3120 [==============================] - 7s 2ms/step - loss: 17474284.0000
Epoch 3/5
3120/3120 [==============================] - 7s 2ms/step - loss: 15024709.0000
Epoch 4/5
3120/3120 [==============================] - 7s 2ms/step - loss: 13736786.0000
Epoch 5/5
3120/3120 [==============================] - 3s 973us/step
R2 for fold 1: 0.820
------------------------------------------------------------------------
Training for fold 2 ...
Epoch 1/5
3120/3120 [==============================] - 7s 2ms/step - loss: 87791712.0000
Epoch 2/5
3120/3120 [==============================] - 7s 2ms/step - loss: 21392250.0000
Epoch 3/5
3120/3120 [==============================] - 7s 2ms/step - loss: 18700814.0000
Epoch 4/5
3120/3120 [==============================] - 7s 2ms/step - loss: 17212228.0000
Epoc

The CV R2 score is 0.839.


In [45]:
# Define the K-fold Cross Validator
kfold = KFold(n_splits=2, shuffle=True)

# K-fold Cross Validation model evaluation
fold_no = 1
scores = []

for cv_train, cv_test in kfold.split(X_train_scaled, y_train):
    model = Sequential(
    [
        tf.keras.layers.Dense(128, activation = 'relu', name='L1'),
        tf.keras.layers.Dense(64, activation = 'relu', name='L4'),
        tf.keras.layers.Dense(1, activation = 'relu', name='L5'),

    ])
    model.compile(loss = MeanSquaredError(), optimizer = tf.keras.optimizers.Adam(learning_rate=0.01))
        
    # Generate a print
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')
    
    model.fit(X_train_scaled.loc[cv_train,:], y_train.loc[cv_train], epochs = 5)

    # Generate generalization metrics
    y_pred = model.predict(X_train_scaled.loc[cv_test,:])
    training_error = r2_score(y_pred,y_train.loc[cv_test])
    
    print(f'R2 for fold {fold_no}: {training_error:.3f}')
    scores.append(training_error)

    # Increase fold number
    fold_no = fold_no + 1
print(f'The CV R2 score is {np.mean(scores):.3f}.')

------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/5
3120/3120 [==============================] - 8s 2ms/step - loss: 26403096.0000
Epoch 2/5
3120/3120 [==============================] - 7s 2ms/step - loss: 12704847.0000
Epoch 3/5
3120/3120 [==============================] - 8s 2ms/step - loss: 10537019.0000
Epoch 4/5
3120/3120 [==============================] - 8s 3ms/step - loss: 9539421.0000
Epoch 5/5
3120/3120 [==============================] - 3s 985us/step
R2 for fold 1: 0.910
------------------------------------------------------------------------
Training for fold 2 ...
Epoch 1/5
3120/3120 [==============================] - 8s 2ms/step - loss: 24516888.0000
Epoch 2/5
3120/3120 [==============================] - 8s 2ms/step - loss: 11567074.0000
Epoch 3/5
3120/3120 [==============================] - 8s 3ms/step - loss: 9785153.0000
Epoch 4/5
3120/3120 [==============================] - 8s 2ms/step - loss: 8748250.0000
Epoch 5

In [59]:
# Define the K-fold Cross Validator
kfold = KFold(n_splits=3, shuffle=True)

# K-fold Cross Validation model evaluation
fold_no = 1
scores = []

for cv_train, cv_test in kfold.split(X_train_scaled, y_train):
    model = Sequential(
    [
        tf.keras.layers.Dense(256, activation = 'relu', name='L1'),
        tf.keras.layers.Dense(128, activation = 'relu', name='L2'),
        tf.keras.layers.Dense(64, activation = 'relu', name='L4'),
        tf.keras.layers.Dense(1, activation = 'relu', name='L5'),

    ])
    model.compile(loss = MeanSquaredError(), optimizer = tf.keras.optimizers.Adam(learning_rate=0.01))
        
    # Generate a print
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')
    
    model.fit(X_train_scaled.loc[cv_train,:], y_train.loc[cv_train], epochs = 5)

    # Generate generalization metrics
    y_pred = model.predict(X_train_scaled.loc[cv_test,:])
    training_error = r2_score(y_pred,y_train.loc[cv_test])
    
    print(f'R2 for fold {fold_no}: {training_error:.3f}')
    scores.append(training_error)

    # Increase fold number
    fold_no = fold_no + 1
print(f'The CV R2 score is {np.mean(scores):.3f}.')

------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/5
4160/4160 [==============================] - 12s 3ms/step - loss: 30435272.0000
Epoch 2/5
4160/4160 [==============================] - 11s 3ms/step - loss: 11258642.0000
Epoch 3/5
4160/4160 [==============================] - 11s 3ms/step - loss: 9594129.0000
Epoch 4/5
4160/4160 [==============================] - 11s 3ms/step - loss: 8490998.0000
Epoch 5/5
2080/2080 [==============================] - 2s 989us/step
R2 for fold 1: 0.872
------------------------------------------------------------------------
Training for fold 2 ...
Epoch 1/5
4160/4160 [==============================] - 11s 3ms/step - loss: 32760426.0000
Epoch 2/5
4160/4160 [==============================] - 11s 3ms/step - loss: 13712922.0000
Epoch 3/5
4160/4160 [==============================] - 11s 3ms/step - loss: 11747593.0000
Epoch 4/5
4160/4160 [==============================] - 11s 3ms/step - loss: 10414745.000

## 4.8 Hyperparameter optimization

## 4.7 Training models & evaluation

In [ ]:
lmbd = 0
model1 = LinearRegression()
model2 = KNeighborsRegressor()
model3 = DecisionTreeRegressor()
model4 = RandomForestRegressor()
model5 = XGBRegressor()
model6 = Sequential(
[
    tf.keras.layers.Dense(2048, activation = 'relu', name='L1', kernel_regularizer=L2(lmbd)),
    tf.keras.layers.Dense(2048, activation = 'relu', name='L2', kernel_regularizer=L2(lmbd)),
    tf.keras.layers.Dense(1, activation = 'relu', name='L7'),

]
)
model6.compile(loss = MeanSquaredError(),optimizer = tf.keras.optimizers.Adam(learning_rate=0.01))
model_pipeline = [model1,  model4, model6, model5,]
model_names = ['Linear Regression', 'RandomForest', 'NeuralNetwork' ,'XGBoost']

scores = {}
for model, model_name in zip(model_pipeline, model_names):
    print('Working with model '+model_name)
    
    # Fitting the model
    if model_name == 'NeuralNetwork':
        model.fit(X_train_scaled, y_train, epochs = 30)
    else:
        model.fit(X_train_scaled, y_train)
    y_pred = model.predict(X_train_scaled)
    training_error = r2_score(y_pred,y_train)
    
    y_pred_cv = model.predict(X_cv_scaled)
    cv_error = r2_score(y_pred_cv,y_cv)
    scores[model_name] = [round(training_error,3), round(cv_error,3)]
    
print(scores)
# We can use the result to choose the best performing model

## 4.7 Hyperparameter search

### 4.7.1 Random Forest

In [ ]:
# from sklearn.model_selection import RandomizedSearchCV

max_depth_choices= [5,10,None]
criterion_choices = ['squared_error','absolute_error']
min_samples_split_choices = [4,10]
min_samples_leaf_choices = [4,10]

random_grid = {'max_depth': max_depth_choices,
'criterion': criterion_choices,
'min_samples_split': min_samples_split_choices,
'min_samples_leaf': min_samples_leaf_choices}

# Instantiate the grid search model object
# estimator -> model to optimize
model = RandomForestRegressor()

#and now more realistic
model = DecisionTreeRegressor()
random_search = RandomizedSearchCV(estimator = model, param_distributions = random_grid, n_iter=12, cv = 5, n_jobs = 8)

random_search.fit(X_train_scaled,y_train)

In [ ]:
random_search.best_params_

In [ ]:
print(f'The best score (R2={random_search.best_score_:.2f}) corresponds to the parameters {random_search.best_params_}.')

The best score (R2=0.94) corresponds to the parameters {'min_samples_split': 4, 'min_samples_leaf': 4, 'max_depth': None, 'criterion': 'squared_error'}.

# 4.7.2 XGBoost

In [13]:
# from sklearn.model_selection import RandomizedSearchCV

param_grid = {
        'max_depth': [3, 7, 11, None],
        'min_child_weight': np.arange(0.0001, 0.5, 0.1),
        'learning_rate': np.arange(0.0005,0.3,0.05),
        'subsample': np.arange(0.01,1.0,0.01),
        'colsample_bylevel': np.round(np.arange(0.1,1.0,0.1)),
        'colsample_bytree': np.arange(0.1,1.0,0.1),
}

# Instantiate the grid search model object
# estimator -> model to optimize
model = XGBRegressor()

random_search = RandomizedSearchCV(estimator = model, param_distributions = param_grid, n_iter=50, cv = 5, n_jobs = 8)

random_search.fit(X_train_scaled,y_train)

RandomizedSearchCV(cv=5,
                   estimator=XGBRegressor(base_score=None, booster=None,
                                          callbacks=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None,
                                          early_stopping_rounds=None,
                                          enable_categorical=False,
                                          eval_metric=None, feature_types=None,
                                          gamma=None, gpu_id=None,
                                          grow_policy=None,
                                          importance_type=None,
                                          interaction_constraints=None,
                                          learning_rate=...
       0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3 , 0.31, 0.32, 0.33,
       0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4 , 0.41, 0.42, 0.43, 0.44,
       0.45, 0.46, 0.47, 0.48, 0.49, 0.5 , 0.51, 0.52, 0.53, 0.54, 0.55,
       0.56, 0.57, 0.58, 0.59, 0.6 , 0.61, 0.62, 0.63, 0.64, 0.65, 0.66,
       0.67, 0.68, 0.69, 0.7 , 0.71, 0.72, 0.73, 0.74, 0.75, 0.76, 0.77,
       0.78, 0.79, 0.8 , 0.81, 0.82, 0.83, 0.84, 0.85, 0.86, 0.87, 0.88,
       0.89, 0.9 , 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98, 0.99])})

In [14]:
random_search.best_params_

{'subsample': 0.52,
 'min_child_weight': 0.4001,
 'max_depth': 11,
 'learning_rate': 0.2505,
 'colsample_bytree': 0.9,
 'colsample_bylevel': 1.0}

In [15]:
random_search.best_score_

0.9610301120274907

In [16]:
model = XGBRegressor(**random_search.best_params_)

XGB_topscore = np.mean(cross_val_score(model, X_train_scaled, y_train, cv=3))

In [17]:
XGB_topscore

0.9571179327361223

In [19]:
model = RandomForestRegressor(**{'min_samples_split': 4, 'min_samples_leaf': 4, 'max_depth': None, 'criterion': 'squared_error'})
np.mean(cross_val_score(model, X_train_scaled, y_train, cv=3))

0.9571853208535602

In [ ]:
# in grid search you are more likely to get really good results in your training set, even with CV
grid_search.best_score_

In [ ]:
np.round(y_pred_cv,0).flatten()

In [ ]:
error =int((abs(np.round(y_pred_cv,0).flatten()-y_cv)/np.round(y_pred_cv,0).flatten()*100).astype(int).values.mean())
std = int((abs(np.round(y_pred_cv,0).flatten()-y_cv)/np.round(y_pred_cv,0).flatten()*100).astype(int).values.std())
print('The error is ',error,'±',2*std)

In [ ]:
# <!> I could try to use log or sqrt of some qualities like cv and more that seem to over emphasize 
# the price of high values.

In [ ]:
(np.round(y_pred_cv,0).flatten()-y_cv).astype(int).values[abs(np.round(y_pred_cv,0).flatten()-y_cv).astype(int).values>6000].mean()

In [ ]:
(X_cv[abs(np.round(y_pred_cv,0).flatten()-y_cv).astype(int).values>6000]).describe().T

In [ ]:
(X_cv[abs(np.round(y_pred_cv,0)-y_cv).astype(int).values<6000]).describe().T

In [ ]:
abs(np.round(y_pred_cv,0)-y_cv).astype(int).values

In [ ]:
ca

In [ ]:
a = 'marchas'

In [ ]:
round(len(X_cv[X_cv[a]==0.])/X_cv.shape[0]*100,3)

In [ ]:
round(len(X_train[X_train[a]==0.])/X_train.shape[0]*100,3)

In [ ]:
X_cv[a]==0